In [13]:
import sqlite3
path = "pizza_app.sqlite"
connection = sqlite3.connect(path)

In [14]:
## Forma mais direta
import sqlite3
connection = sqlite3.connect("pizza_app.sqlite")

In [15]:
from sqlite3 import Error

def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)

        # commit necessário para alterações no banco
        connection.commit() ###

        print(f"Query executada.")
        if cursor.rowcount != -1:
            print(f"{cursor.rowcount} linha(s) afetadas")

    except Error as e:
        print(f"Erro: '{e}'")

In [16]:
# Cria a tabela produto #
create_produto_table = \
"""CREATE TABLE produto (
    id_produto INTEGER PRIMARY KEY AUTOINCREMENT,
    tipo VARCHAR(50),
    desc_item VARCHAR(100),
    vl_preco DECIMAL(10, 2)
);"""

execute_query(connection, create_produto_table)
#########################

# Cria a tabela pedido #
create_pedido_table = \
"""CREATE TABLE pedido (
    id_pedido INTEGER PRIMARY KEY AUTOINCREMENT,
    dt_pedido DATE,
    fl_ketchup BOOLEAN,
    desc_uf CHAR(2),
    txt_recado TEXT
);"""

execute_query(connection, create_pedido_table)
#########################

# Cria a tabela item_pedido #
create_item_pedido_table = \
"""CREATE TABLE item_pedido (
    id_pedido INT NOT NULL,
    id_produto INT NOT NULL,
    quantidade INT NOT NULL,
    PRIMARY KEY (id_pedido, id_produto),
    FOREIGN KEY (id_pedido) REFERENCES pedido(id_pedido),
    FOREIGN KEY (id_produto) REFERENCES produto(id_produto)
);"""
execute_query(connection, create_item_pedido_table)
#########################

Erro: 'table produto already exists'
Erro: 'table pedido already exists'
Erro: 'table item_pedido already exists'


In [17]:
## Inserindo registros manualmente

# Inserindo produto #
insert_produto = \
"""INSERT INTO
produto (tipo, desc_item, vl_preco)
VALUES
('ingrediente', 'camarão', 6),
('massa', 'tradicional', 9.25),
('borda', 'tradicional', 0),
('queijo', 'muçarela', 4),
('bebida', 'refrigerante', 5);
"""
execute_query(connection, insert_produto)
######################

# Inserindo pedido
insert_pedido = \
"""INSERT INTO
pedido (dt_pedido, fl_ketchup, desc_uf, txt_recado)
VALUES
('2023-06-01', TRUE, 'MG', 'Capricha no queijo!');
"""
execute_query(connection, insert_pedido)
######################

Query executada.
5 linha(s) afetadas
Query executada.
1 linha(s) afetadas


In [18]:
# Inserindo item_pedido
itens = (
    {'id_pedido': 1, 'id_produto': 2, 'qtd': 1},
    {'id_pedido': 1, 'id_produto': 3, 'qtd': 1},
    {'id_pedido': 1, 'id_produto': 1, 'qtd': 1},
    {'id_pedido': 1, 'id_produto': 4, 'qtd': 2},
    {'id_pedido': 1, 'id_produto': 5, 'qtd': 3}
)

insert_item_pedido = \
"""INSERT INTO item_pedido (id_pedido, id_produto, quantidade)
VALUES(:id_pedido, :id_produto, :qtd);"""

cursor = connection.cursor()
cursor.executemany(insert_item_pedido, itens)
connection.commit() # necessário para inserções
cursor.close()

In [19]:
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()

        return result
    except Error as e:
        print(f"Erro: '{e}'")

In [20]:
tabela = 'produto'
query = f"SELECT * FROM {tabela}"
resultado = execute_read_query(connection, query)

print(f"Tabela: {tabela}")
for res in resultado:
    print(res)

Tabela: produto
(6, 'ingrediente', 'camarão', 6)
(7, 'massa', 'tradicional', 9.25)
(8, 'borda', 'tradicional', 0)
(9, 'queijo', 'muçarela', 4)
(10, 'bebida', 'refrigerante', 5)


In [21]:
select_table_names = \
"SELECT name FROM sqlite_schema WHERE type='table';"
tables = execute_read_query(connection, select_table_names)
print(tables, '\n')

for table in tables:
    select_all = f"SELECT * FROM {table[0]}"
    res = execute_read_query(connection, select_all)
    print(f"{table[0]}: {res}")

[('produto',), ('sqlite_sequence',), ('pedido',), ('item_pedido',)] 

produto: [(6, 'ingrediente', 'camarão', 6), (7, 'massa', 'tradicional', 9.25), (8, 'borda', 'tradicional', 0), (9, 'queijo', 'muçarela', 4), (10, 'bebida', 'refrigerante', 5)]
sqlite_sequence: [('produto', 10), ('pedido', 2)]
pedido: [(2, '2023-06-01', 1, 'MG', 'Capricha no queijo!')]
item_pedido: [(1, 2, 1), (1, 3, 1), (1, 1, 1), (1, 4, 2), (1, 5, 3)]


In [22]:
execute_query(connection, "DELETE FROM item_pedido;")
execute_query(connection, "DELETE FROM pedido;")
execute_query(connection, "DELETE FROM produto;")

Query executada.
5 linha(s) afetadas
Query executada.
1 linha(s) afetadas
Query executada.
5 linha(s) afetadas
